In [25]:
import pandas as pd
import requests
import os

In [26]:
df = pd.read_csv("subtitles_collected.csv")

In [27]:
df['subtitle'].value_counts()

subtitle
SDH       2863
EN         281
SDH_EN     250
Name: count, dtype: int64

In [28]:
df['subtitle'].isnull().sum()

182

In [24]:
2792+281+250+182

3505

In [16]:
df[df['subtitle'] == 'SDH']['sdh_sub_lang'].value_counts()

sdh_sub_lang
en-us      2546
en-gb       172
pt-br        57
te-in        49
hi-in        37
en-in        30
es-419       29
en           27
en-ca        26
ta-in        23
fr-fr        21
en-au        13
es-mx        12
ja-jp        11
es-es         9
th-th         7
it-it         6
zh-hant       5
pt-pt         3
ko-kr         3
ru-ru         3
en-za         3
ar-sa         2
es-cl         2
es-co         2
sv-se         2
de-de         2
sv-sv         1
es-ar         1
en-ie         1
fil-ph        1
ar-001        1
nl-nl         1
en-ng         1
bg-bg         1
fa-ir         1
fi-fi         1
tr-tr         1
Name: count, dtype: int64

In [19]:
df[(~df['sdh_sub_lang'].isnull()) & (df['sdh_sub_lang'].str.contains('en-gb'))]

,file,title,entityType,runtimeSeconds,synopsis,rating_count,rating,en_type,en_url,sdh_sub_lang,url,subtitle
54,1086_Among_the_Living,Among the Living,Movie,5124.0,Stranded in the countryside in the aftermath o...,233.0,3.5,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/e313/f465/...,SDH
59,1091_Bird_Catcher_The,"Bird Catcher, The",Movie,6020.0,Norway 1942 - Fleeing the Nazi round-up of Jew...,205.0,4.1,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/3065/dc5d/...,SDH
78,1131_Dil_Chahta_Hai,Dil Chahta Hai,Movie,11103.0,"Three superstars Aamir Khan, Saif Ali Khan and...",75.0,4.1,subtitle,https://cf-timedtext.aux.pv-cdn.net/5fd9/8625/...,en-gb,https://cf-timedtext.aux.pv-cdn.net/009f/c0ab/...,SDH
131,1230_Lusitania,Lusitania,Movie,5438.0,"The sinking of the Lusitania, one of the world...",31.0,3.9,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/7404/6b0a/...,SDH
132,1233_Loving_Miss_Hatto,Loving Miss Hatto,Movie,5459.0,The story of one of Britain's most famous pian...,19.0,3.5,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/82be/981e/...,SDH
...,...,...,...,...,...,...,...,...,...,...,...,...
3433,283_Brighton,Brighton,Movie,5260.0,A typical day in Brighton; two working class c...,112.0,3.7,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/5424/a7f2/...,SDH
3466,376_Firenado,Firenado,Movie,4738.0,Four scientists uncover a ground-breaking tech...,14.0,2.9,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/f6c8/bb2a/...,SDH
3536,702_Dhruva,Dhruva,Movie,8420.0,An ancient Martial art form of Tamilnadu is in...,2.0,3.1,NaN,NaN,en-gb,https://cf-timedtext.aux.pv-cdn.net/2925/7a19/...,SDH
3537,703_Local_Boy,Local Boy,Movie,8421.0,An ancient Martial art form of Tamilnadu is in...,5.0,2.5,subtitle,https://cf-timedtext.aux.pv-cdn.net/78ac/2cc4/...,en-gb,https://cf-timedtext.aux.pv-cdn.net/132d/c215/...,SDH


In [2]:
import sys
sys.path.append("../")

In [3]:
from consts import CLEAN_SCRAPE_DIR, BEFORE_2010_DIR, IN_2010S, AFTER_2020
TARGETS = [CLEAN_SCRAPE_DIR, BEFORE_2010_DIR, IN_2010S, AFTER_2020]

In [4]:
if not os.path.exists("./subtitles"): os.mkdir("./subtitles")

for TARGET in TARGETS:
    df = pd.read_csv(f"../4_parse_xrays/parsed_xrays/{TARGET}_sub_movies_with_xrays.csv")
    sdh = df[~df['sdh_sub_lang'].isnull()]
    print(sdh.shape)

(2561, 11)
(121, 11)
(211, 11)
(220, 11)


In [27]:
if not os.path.exists("./subtitles"): os.mkdir("./subtitles")

for TARGET in TARGETS:
    df = pd.read_csv(f"../4_parse_xrays/parsed_xrays/{TARGET}_sub_movies_with_xrays.csv")
    sdh = df[~df['sdh_sub_lang'].isnull()]

    if not os.path.exists(f"./subtitles/{TARGET}"): os.mkdir(f"./subtitles/{TARGET}")

    for index, row in sdh.iterrows():
        print(TARGET, index)
        directory = f"./subtitles/{TARGET}/{row['file']}"
        if not os.path.exists(directory): os.mkdir(directory)

        sub_file_name = f"./subtitles/{TARGET}/{row['file']}/{row['file']}.ttml2"
        if os.path.exists(sub_file_name): continue
        
        sub = requests.get(row['url'])
        with open(sub_file_name, "wb") as f:
            f.write(sub.content) 

com 0
com 1
com 3
com 4
com 5
com 6
com 7
com 8
com 9
com 11
com 12
com 13
com 14
com 15
com 16
com 17
com 18
com 19
com 20
com 21
com 22
com 23
com 24
com 25
com 26
com 27
com 28
com 29
com 30
com 31
com 33
com 34
com 35
com 36
com 37
com 38
com 39
com 40
com 41
com 42
com 43
com 44
com 45
com 46
com 47
com 49
com 50
com 51
com 52
com 53
com 54
com 55
com 56
com 58
com 59
com 61
com 62
com 63
com 64
com 65
com 66
com 67
com 68
com 69
com 70
com 71
com 72
com 73
com 74
com 75
com 76
com 78
com 79
com 80
com 81
com 82
com 83
com 84
com 85
com 86
com 87
com 88
com 89
com 90
com 91
com 92
com 93
com 94
com 96
com 97
com 98
com 99
com 100
com 102
com 103
com 104
com 105
com 106
com 107
com 108
com 109
com 111
com 112
com 113
com 114
com 115
com 116
com 117
com 118
com 119
com 120
com 121
com 122
com 123
com 124
com 125
com 126
com 127
com 128
com 129
com 130
com 131
com 132
com 133
com 135
com 136
com 137
com 138
com 139
com 140
com 141
com 143
com 145
com 146
com 147
com 148
com 150
com 1

KeyboardInterrupt: 